In [1]:
import codecs
import random
import numpy as np
import pandas as pd

In [2]:
enemy_types = ['zombie', 'shotgun zombie', 'rifleman zombie', 'chaingunner', 'imp', 'pinky', 'lostsoul', 'hellknight', 'cacodemon', 'mancubus']
item_types = ['medikit', 'little medikit', 'health bonus', 'armour', 'armour bonus', 'shells', 'clip', 'rocket launcher']
corpse_types = ['dead ' + e for e in enemy_types]
pos_types = ['at left', 'left-ahead', 'ahead', 'right-ahead', 'at right']
weapon_types = ['pistol', 'shotgun', 'chainsaw', 'double-barrel', 'chaingun', 'rocket launcher']
dmg = {'pistol':1, 'shotgun':2, 'chainsaw':3, 'double-barrel':4, 'chaingun':3, 'rocket launcher':7}
hp = {'zombie':2, 'shotgun zombie':4, 'rifleman zombie':2, 'chaingunner':6, 'imp':4, 'pinky':7, 'lostsoul':6, 'hellknight':14, 'cacodemon':10, 'mancubus':16}

In [3]:
action_count_max = 10

In [4]:
full_writable_text = ''

for story in range(2000):
    enemy_d = {}
    item_d = {}
    enemy_count = int(15*np.random.rand()**3)
    item_count = int(np.random.rand()*12)
    corpse_count = int(14*np.random.rand()**2)
    action_count = 0

    for task in ['kill all']:
        df = []
        descripion = 'description: '
        current_weapon = random.choice(weapon_types)
        obj_id = 0
        for i in range(enemy_count):
            enemy_cur = random.choice(enemy_types)
            pos_cur = random.choice(pos_types)
            rec = {'obj_name':enemy_cur, 'obj_type':'enemy', 'pos_cur':pos_cur, 'obj_id':obj_id}
            obj_id += 1
            df.append(rec)
            descripion += f"{enemy_cur} {pos_cur}, "
        for i in range(item_count):
            item_cur = random.choice(item_types)
            pos_cur = random.choice(pos_types)
            rec = {'obj_name':item_cur, 'obj_type':'item', 'pos_cur':pos_cur, 'obj_id':obj_id}
            obj_id += 1
            df.append(rec)
            descripion += f"{item_cur} {pos_cur}, "
        for i in range(corpse_count):
            corpse_cur = random.choice(corpse_types)
            pos_cur = random.choice(pos_types)
            rec = {'obj_name':corpse_cur, 'obj_type':'corpse', 'pos_cur':pos_cur, 'obj_id':obj_id}
            obj_id += 1
            df.append(rec)
            descripion += f"{corpse_cur} {pos_cur}, "
        descripion = descripion[:-2]
        df = pd.DataFrame(df)
        if df.shape[0]<2:
            continue

        plan = []
        if task == 'kill all':
            while action_count<action_count_max:
                if df[(df['obj_type']=='enemy') & (df['pos_cur']=='ahead')].shape[0]>0:
                    aim = df[(df['obj_type']=='enemy') & (df['pos_cur']=='ahead')].iloc[0]
                    shots = int(np.ceil(hp[aim['obj_name']]/dmg[current_weapon]))
                    action_count += shots
                elif df[(df['obj_type']=='enemy') & (df['pos_cur']=='left-ahead')].shape[0]>0:
                    aim = df[(df['obj_type']=='enemy') & (df['pos_cur']=='left-ahead')].iloc[0]
                    shots = int(np.ceil(hp[aim['obj_name']]/dmg[current_weapon]))
                    action_count += shots
                    plan += ['turn left']
                elif df[(df['obj_type']=='enemy') & (df['pos_cur']=='right-ahead')].shape[0]>0:
                    aim = df[(df['obj_type']=='enemy') & (df['pos_cur']=='right-ahead')].iloc[0]
                    shots = int(np.ceil(hp[aim['obj_name']]/dmg[current_weapon]))
                    action_count += shots
                    plan += ['turn right']
                elif df[(df['obj_type']=='enemy') & (df['pos_cur']=='at left')].shape[0]>0:
                    aim = df[(df['obj_type']=='enemy') & (df['pos_cur']=='at left')].iloc[0]
                    shots = int(np.ceil(hp[aim['obj_name']]/dmg[current_weapon]))
                    action_count += shots
                    plan += ['turn left','turn left']
                elif df[(df['obj_type']=='enemy') & (df['pos_cur']=='at right')].shape[0]>0:
                    aim = df[(df['obj_type']=='enemy') & (df['pos_cur']=='at right')].iloc[0]
                    shots = int(np.ceil(hp[aim['obj_name']]/dmg[current_weapon]))
                    action_count += shots
                    plan += ['turn right','turn right']
                else:
                    break

                df = df[df.obj_id!=aim.obj_id]
                if current_weapon == 'chainsaw':
                    plan += ['go ahead']
                plan += ['fire']*shots
            #print(descripion)
            #print(plan)
            plan = str(plan)[1:-1].replace("'", '')
            string_to_write = f'{descripion} weapon: {current_weapon}, task: {task} plan: {plan}<END>\n'
            #print(string_to_write)
            full_writable_text += string_to_write
full_writable_text

'description: cacodemon right-ahead, dead chaingunner at left, dead imp ahead weapon: chainsaw, task: kill all plan: turn right, go ahead, fire, fire, fire, fire<END>\ndescription: zombie ahead, cacodemon right-ahead, rifleman zombie right-ahead, armour at right, health bonus at right, armour left-ahead, rocket launcher at right, health bonus right-ahead, little medikit at left, clip at left, dead cacodemon at left, dead cacodemon at left, dead imp at left, dead lostsoul at left weapon: chainsaw, task: kill all plan: go ahead, fire, turn right, go ahead, fire, fire, fire, fire, turn right, go ahead, fire<END>\ndescription: imp left-ahead, hellknight at left, armour bonus left-ahead, little medikit right-ahead, medikit ahead, armour left-ahead, medikit at right, armour at left, armour bonus right-ahead, clip ahead, armour bonus left-ahead, dead cacodemon at right, dead zombie left-ahead, dead shotgun zombie at left weapon: rocket launcher, task: kill all plan: turn left, fire, turn left

In [5]:
descripion

'description: armour bonus at right, health bonus right-ahead, little medikit right-ahead, shells at right, medikit ahead, armour bonus at right, clip right-ahead, rocket launcher at right, medikit right-ahead, clip left-ahead, dead hellknight right-ahead'

In [6]:
with codecs.open('data/toy_text_doom_tasks.txt', 'w', 'utf8') as f:
    f.write(full_writable_text)